In [ ]:
%%writefile Finanzas.py
import numpy as np
import streamlit as st
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

st.write(''' # Predicción de Finanzas Personales ''')
st.image("dinero.jpg", caption="Control financiero mensual.")

st.header('Datos de evaluación')

def user_input_features():
  Gastos = st.number_input('¿Cuánto gastas al mes?', min_value=0, step=100)
  Reinversion = st.number_input('¿Cuánto reinviertes al mes?', min_value=0, step=100)
  GananciaReinversion = st.number_input('¿Cuánto ganas de reinversiones al mes?', min_value=0, step=100)
  IngresoActual = st.number_input('¿Cuánto ganas actualmente al mes?', min_value=0, step=100)

  user_data = {'Gastos': Gastos,
               'Reinversion': Reinversion,
               'GananciaReinversion': GananciaReinversion,
               'IngresoActual': IngresoActual}

  return pd.DataFrame(user_data, index=[0])

df = user_input_features()

fin = pd.read_csv("Finanzas.csv")
X = fin[["Gastos","Reinversion","GananciaReinversion","IngresoActual"]]
Y = (fin["IngresoActual"] + fin["GananciaReinversion"]) >= fin["Gastos"]
Y = Y.astype(int)

modeloA = DecisionTreeClassifier(max_depth=5, criterion='entropy', min_samples_leaf=5, random_state=0)
modeloA.fit(X, Y)

modeloB = DecisionTreeRegressor(max_depth=5)
fin["IngresoNecesario"] = fin["Gastos"] - fin["GananciaReinversion"]
modeloB.fit(fin[["Gastos","GananciaReinversion"]], fin["IngresoNecesario"])

predA = modeloA.predict(df)
predB = modeloB.predict(df[["Gastos","GananciaReinversion"]])[0]

st.subheader('Predicción A: ¿Te alcanza el dinero?')
if predA == 1:
  st.write('✔ Sí te alcanza para tus gastos.')
else:
  st.write('❌ No te alcanza con tus ingresos actuales.')

st.subheader('Predicción B: Ingreso mínimo recomendado')
ingreso_20 = predB * 1.20
st.write(f"Debes ingresar al mes al menos: **${ingreso_20:.2f} MXN**")

st.info(f"Para mantener un equilibrio bien balanceado entre tus gastos e ingresos, "
        f"y además tener un 20% extra para ahorrar, necesitas ingresar al menos "
        f"**${ingreso_20:.2f} MXN** al mes.")

st.subheader("Clasificación de tu nivel de gasto")

g = df["Gastos"].iloc[0]
if g < 3000:
  st.write("Gasto bajo")
elif g < 9000:
  st.write("Gasto normal")
elif g < 15000:
  st.write("Gasto un poco alto")
else:
  st.write("Gasto muy alto")